# Connect to DB, Fetch data

In [1]:
import pandas as pd
import numpy as np
# specify connection to database
import psycopg2
connection = psycopg2.connect(
    host="nursedash-prod.cuzi2kducsnv.us-east-1.rds.amazonaws.com",
    database="nursedash",
    user="external_analyst",
    password="vWHYpF9CNtC9KWBG7sZ5JvX9")

### <font color = green> all time to chicago time, No withdrawn info

In [2]:
df = pd.read_sql_query("""

SELECT  sa.id, sa.user_id, sa.shift_id, f.id AS facility_id, sa."withdrawnInfo" -> 'initiator' as withdrawnInfo_value,
sa."status", sa."prevStatus", "s"."status" AS "s_status", s."facility_id", "s"."description" AS "shift_description",
"s"."assigned_nurse_id", s."net_pay", "s"."unit" AS "s_unit",s."type", sa."hasNurseCheckEvent",
"s"."qualifications" AS "s_qualifications", "s"."breakTime" AS "s_breakTime", sa."withdrawnInfo",
"f"."name" AS "f_name", f."segmentName", f."areaName", 
timezone('America/Chicago', t."currentCheckInTime") as t_checkin,
timezone('America/Chicago', sa."createdAt") as sa_create,
timezone('America/Chicago', u."approvedAt") as u_approve,
timezone('America/Chicago', u."createdAt") as u_create,
timezone('America/Chicago', sa."statusUpdatedAt") as sa_statusUpdate,
timezone('America/Chicago', timezone('UTC', s.start_time)) AS "Start_Time" 
FROM shifts s
Left JOIN "TimeChangeRequests" t ON t."shiftId" = s.id
INNER JOIN shift_applications sa ON s.id = sa.shift_id
INNER JOIN facilities f ON s.facility_id = f.id
INNER JOIN users u ON sa.user_id = u.id

""", con = connection)

### <font color = green> these features was done in sql before

### U_create2U_approve, in hours

In [3]:
from datetime import datetime
df['U_create2U_approve'] = df.apply(lambda row: (row['u_approve'] - row['u_create']).total_seconds()/3600, 
                                    axis = 1)

### U_approve2now, in month

In [4]:
df['U_approve2now'] = df.apply(lambda row: (datetime.now() - row['u_approve']).total_seconds()/2629746, 
                               axis = 1)

### CW_Time2Start_Time

In [5]:
# CW_Time2Start_Time
df['CW_Time2Start_Time'] = df.apply(lambda row: (row['Start_Time'] - row['sa_statusupdate']).total_seconds()/3600 if str(row['Start_Time'])[:2] != '00' else 9999, 
                                         axis = 1)

### Checkin time 2Start_Time, In minutes

In [6]:
df['checkin_Time2Start_Time'] = df.apply(lambda row: (row['Start_Time'] - row['t_checkin']).total_seconds()/60 if str(row['Start_Time'])[:2] != '00' else 99, 
                                         axis = 1)
# # fill na as 0
# df['checkin_Time2Start_Time'] = df['checkin_Time2Start_Time'].fillna(0)

# Data Cleaning

In [7]:
# sort by time that shift was created
df = df.sort_values(by = 'sa_create').reset_index(drop = True)

# Feature Engineering

### Target: urgent withdrawn as 1

In [8]:
def CW_by_nurse(row):
    if row['status']=='withdrawn' and row['prevStatus'] == 'confirmed':
        if row['withdrawninfo_value'] == 'nurse':
            if row['CW_Time2Start_Time'] <= 24:
                return 1
            else:
                return 0
        else:
            return 0
    else:
        return 0
    
df['target'] = df.apply (lambda row: CW_by_nurse(row), axis=1)

In [9]:
df['target'].value_counts()

0    195581
1      6162
Name: target, dtype: int64

### find extreme UCW rate facility from Top 30

In the TOP 30 facilities, Ohio Living Breckenridge Village,The Weils and Park Manor of South Belt has the highest UCW rate.

Whileas, The Hallmark and Village of Meyerland has the lowest UCW rate

In [10]:
df['f_highrate'] = df.apply(lambda row:1 if row['f_name'] in ['Ohio Living Breckenridge Village',
                                                              'The Weils', 
                                                              'Park Manor of South Belt'] else 0, axis = 1)
df['f_lowrate'] = df.apply(lambda row:1 if row['f_name'] in ['The Hallmark', 
                                                              'Village of Meyerland'] else 0, axis = 1)

###  Count previsous shift application

In [11]:
# track by using dictionary
count_prev_SA = []
test = df[['user_id', 'sa_create']]

# create SA dictionary, set all value = 0 -> dramatically reduce computational cost
uid_library = list(pd.unique(test['user_id']))
sa_dict = {} 
for uid in uid_library:
    sa_dict.update({uid: 0}) 
    
for i, v in enumerate(test['user_id']):
    sa_dict[v] += 1
    count_prev_SA.append(sa_dict[v]-1)
 
df['count_prev_SA'] = count_prev_SA

###  Count previsous urgent withdrawns

In [12]:
# track by using dictionary
count_prev_CW = []
test = df[['user_id', 'count_prev_SA', 'sa_create', 'target']]

# create cw dictionary, set all value = 0 -> dramatically reduce computational cost
uid_library = list(pd.unique(test['user_id']))
cw_dict = {} 
for uid in uid_library:
    cw_dict.update({uid: 0}) 
    
# fill dictionary and fill cw count
for i, v in enumerate(test['user_id']):
    if test['target'][i] == 1:
        cw_dict[v] += 1
        count_prev_CW.append(cw_dict[v]-1)
    else:
        count_prev_CW.append(cw_dict[v])
        
df['count_prev_CW'] = count_prev_CW

### Count previsous urgent withdrawns/Count previsous shift applications

In [13]:
df['prev_CW/SA_rate'] = df['count_prev_CW']/df['count_prev_SA']

# fill nan with 0, happend bc 0/0, meaning rate = 0
df['prev_CW/SA_rate'] = df['prev_CW/SA_rate'].fillna(0)

### Previous withdrawn times previous apply

In [14]:
df['prev_CW x SA_rate'] = df['count_prev_CW']*df['count_prev_SA']

### 2 Type dummy: RN, LVN + LPN , Rest


In [15]:
df['type_RN'] = df.apply(lambda row:1 if row['type'] == 'RN' else 0, axis = 1)
df['type_LVN+LPN'] = df.apply(lambda row: 1 if row['type'] == 'LVN' or row['type'] == 'LPN' else 0, axis = 1)

In [16]:
df['type_STNA'] = df.apply(lambda row:1 if row['type'] == 'STNA' else 0, axis = 1)

In [17]:
df['type_CNA'] = df.apply(lambda row:1 if row['type'] == 'CNA' else 0, axis = 1)

### 1 SegementName dummy: Senior Living, Healthcare

In [18]:
def create_segmentName_dummy(row):
    if row['segmentName']=='Senior Living':
        return 1
    else:
        return 0
    
df['segmentName_d'] = df.apply(lambda row: create_segmentName_dummy(row), axis=1)

### 5 Facility Area Name Dummy

In [19]:
df = pd.concat([df, pd.get_dummies(df[['areaName']])], axis=1)

### RS

In [20]:
test = df[['id', 'user_id', 'CW_Time2Start_Time', 'target',
           'status','prevStatus', 'withdrawninfo_value', 
           'checkin_Time2Start_Time', 's_status', 'Start_Time']]
test['count'] = np.ones(len(test))
# sort by start time and user id
test = test.sort_values(by = ['user_id', 'Start_Time'], ascending = True).reset_index(drop = True)

# create realibilty score dictionary
uid_library = list(pd.unique(test['user_id']))
Rscore_dict = {}
for uid in uid_library:
    Rscore_dict.update({uid:40})
    
# create Rs list to join with dataframe
RS_list = []
# fill Rs list
for i, v in enumerate(test['user_id']):
    ## record RS before event happen
    RS_list.append(Rscore_dict[v])
    
    ## calculate next application's RS
    
    # if UCW
    if test['status'][i] == 'withdrawn' and test['prevStatus'][i] == 'confirmed':
        # ucw < 5
        if test['CW_Time2Start_Time'][i] < 5: 
            Rscore_dict[v] -= 40
        # 5 < ucw < 12
        elif test['CW_Time2Start_Time'][i] < 12:
            Rscore_dict[v] -= 20
        # 12 < ucw < 24
        elif test['CW_Time2Start_Time'][i] < 24:
            Rscore_dict[v] -= 10
        # 24 < ucw < 48
        elif test['CW_Time2Start_Time'][i] < 48:
            Rscore_dict[v] -= 5
        # ucw > 48
        else:
            Rscore_dict[v] -= 3
    # if not ucw -> check if check in on time/early/late    
    else:
        if test['checkin_Time2Start_Time'][i] > 2:
            Rscore_dict[v] += 10
        else:
            Rscore_dict[v] += 3
            
test['reliability_score'] = RS_list

# merge with original df
test = test[['id', 'reliability_score']]


<ipython-input-20-4a007819ee95>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['count'] = np.ones(len(test))


In [21]:
RS_list

[40,
 43,
 46,
 49,
 52,
 40,
 43,
 46,
 49,
 52,
 55,
 58,
 61,
 64,
 67,
 70,
 73,
 76,
 40,
 43,
 46,
 49,
 52,
 55,
 58,
 61,
 64,
 67,
 70,
 73,
 40,
 43,
 46,
 49,
 52,
 55,
 58,
 61,
 64,
 67,
 70,
 73,
 76,
 79,
 82,
 85,
 88,
 91,
 94,
 97,
 100,
 40,
 43,
 46,
 49,
 52,
 55,
 58,
 61,
 64,
 67,
 70,
 40,
 43,
 46,
 49,
 52,
 55,
 58,
 61,
 64,
 67,
 70,
 73,
 76,
 79,
 82,
 85,
 88,
 91,
 40,
 40,
 40,
 43,
 40,
 40,
 43,
 46,
 49,
 52,
 55,
 58,
 61,
 64,
 67,
 70,
 73,
 76,
 79,
 82,
 85,
 88,
 91,
 94,
 97,
 100,
 103,
 106,
 109,
 112,
 115,
 118,
 121,
 124,
 127,
 130,
 133,
 136,
 139,
 142,
 145,
 148,
 151,
 154,
 157,
 160,
 163,
 166,
 169,
 172,
 175,
 178,
 181,
 184,
 187,
 190,
 193,
 196,
 199,
 202,
 205,
 208,
 211,
 214,
 217,
 220,
 223,
 226,
 229,
 232,
 235,
 238,
 235,
 238,
 241,
 244,
 247,
 250,
 253,
 256,
 259,
 262,
 265,
 268,
 271,
 274,
 277,
 280,
 283,
 286,
 289,
 292,
 295,
 298,
 301,
 304,
 307,
 310,
 313,
 316,
 319,
 322,
 325,
 328,


In [22]:
def RS(row):
    if row['reliability_score'] > 200:
        return 200
    elif row['reliability_score'] < 0:
        return -100
    else:
        return row['reliability_score']
test['reliability_score'] = test.apply(lambda row: RS(row), axis = 1)

In [23]:
test['reliability_score'].describe()

count    201743.000000
mean        124.170108
std          80.715820
min        -100.000000
25%          61.000000
50%         139.000000
75%         200.000000
max         200.000000
Name: reliability_score, dtype: float64

In [24]:
df1 = df.merge(test, on = 'id', how = 'left')

### keep only prevstatus or status = confirmed

In [25]:
df2 = df1[df1.apply(lambda row: (row['prevStatus'] == 'confirmed') or (row['status'] == 'confirmed'), axis = 1)].reset_index(drop = True)

In [26]:
# how many left
df2.shape

(82470, 52)

### How many previsous urgent withdrawns in a month

### <font color = green>output feature dataframe

In [27]:
df2.columns

Index(['id', 'user_id', 'shift_id', 'facility_id', 'withdrawninfo_value',
       'status', 'prevStatus', 's_status', 'facility_id', 'shift_description',
       'assigned_nurse_id', 'net_pay', 's_unit', 'type', 'hasNurseCheckEvent',
       's_qualifications', 's_breakTime', 'withdrawnInfo', 'f_name',
       'f_short_name', 'segmentName', 'areaName', 't_checkin', 's_create',
       'sa_create', 'u_approve', 'u_create', 'sa_statusupdate', 'Start_Time',
       'U_create2U_approve', 'U_approve2now', 'CW_Time2Start_Time',
       'checkin_Time2Start_Time', 'target', 'f_highrate', 'f_lowrate',
       'count_prev_SA', 'count_prev_CW', 'prev_CW/SA_rate',
       'prev_CW x SA_rate', 'type_RN', 'type_LVN+LPN', 'type_STNA', 'type_CNA',
       'segmentName_d', 'areaName_Austin', 'areaName_Cincinnati',
       'areaName_DFW', 'areaName_Houston', 'areaName_Northeast Ohio',
       'areaName_San Antonio', 'reliability_score'],
      dtype='object')

In [28]:
feature_df = df2[['id', 'user_id', 'shift_id', 'prev_CW/SA_rate', 'status', 'U_approve2now','prev_CW x SA_rate',
                 'type_RN', 'type_LVN+LPN', 'type_STNA', 'segmentName_d', 'areaName', 'type', 
                 'net_pay', 'target', 'sa_create', 'Start_Time', 'areaName_Austin', 'areaName_Cincinnati',
                 'areaName_DFW', 'areaName_Houston', 'areaName_Northeast Ohio', 'areaName_San Antonio', 
                 'count_prev_SA', 'count_prev_CW','f_highrate','f_lowrate','type_CNA','reliability_score']]
# feature_df.to_csv('model_data.csv')

In [29]:
feature_df.shape

(82470, 29)

# Logistic Regression

In [30]:
# feature column
features_colname = ['net_pay', 'prev_CW/SA_rate', 'U_approve2now', 
                    'prev_CW x SA_rate', 'type_RN', 'type_LVN+LPN', 'segmentName_d',  'count_prev_SA', 
                    'count_prev_CW', 'areaName_Northeast Ohio', 'areaName_Houston',
                    'areaName_DFW', 'areaName_Austin', 'areaName_San Antonio'] 



from datetime import date, timedelta
#create a datetime object for tomorrow
tmrrw = date.today() + timedelta(days=1)
#create a string object for tomorrow
tomorrow = str(tmrrw.year) + '-' + str(tmrrw.month) + '-' + str(tmrrw.day)

### Logistic Regression Body(preprocessing - classification report)

In [31]:
###################
### import data ###
###################
import pandas as pd
import numpy as np
# df = pd.read_csv('model_data.csv').drop(columns = ['Unnamed: 0'])
# some values are nan, mark it as -1
df = feature_df
df['U_approve2now'] = df['U_approve2now'].fillna(-1)

######################
### data prepration###
######################

## output: applications after today

# convert to datetime for conditonal selection
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
# sort by start time -> for slicing
df = df.sort_values(by = 'Start_Time') 
# record as realdata
realdata = df[df['Start_Time'].apply(lambda x: x > pd.to_datetime(tomorrow))]
# record predction output rows, don't include it in tran test validation
realdata_len = realdata.shape[0]
# only keep status = confirmed
realdata = realdata[realdata['status'] == 'confirmed']

## Validation set: 1000 recently records
# slice, dont include realdata
validation = df[-1000-realdata_len : -realdata_len] # slice
y_valid = validation['target'] # prep y
x_valid = validation[features_colname] # prep x

## Train test: main dataset - validation set - output(realdata) set
traintest = df[:-1000-realdata_len] # slice 
X = traintest[features_colname] # prep x
y = traintest['target'] # prep y
# set test, train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)


###################
### train logit ###
###################
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
# assign less punlishment for classifying 0 as 1 -> find more 1's
weights = {0:1, 1:10}
# class_weight = 'balanced': automatically adjust weights inversely proportional to class frequencies in the input data
logit = LogisticRegression(solver = 'lbfgs', max_iter=100000, class_weight = weights)
logit.fit(X_train, y_train)


#############
# Threshold #
#############
from sklearn.metrics import roc_curve
from numpy import sqrt
from numpy import argmax
# predict probabilities
yhat = logit.predict_proba(X_test)
# keep probabilities for the positive outcome only
yhat = yhat[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(y_test,yhat)
# calculate the g-mean for each threshold
gmeans = sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = argmax(gmeans)
lower_limiter = thresholds[ix]

# search thresholds for imbalanced classification
from numpy import arange
from sklearn.datasets import make_classification
from sklearn.metrics import f1_score
# apply threshold to positive probabilities to create labels
def to_labels(pos_probs, threshold):
    return (pos_probs >= threshold).astype('int')
# predict probabilities
yhat = logit.predict_proba(X_test)
# keep probabilities for the positive outcome only
probs = yhat[:, 1]
# define thresholds
thresholds = arange(0, 1, 0.001)
# evaluate each threshold
scores = [f1_score(y_test, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = argmax(scores)
higher_limiter = thresholds[ix]


#########################
### train test result ###
#########################
from sklearn.metrics import classification_report, confusion_matrix
y_pred = logit.predict(X_test)
Traintest_CM = confusion_matrix(y_test, y_pred)
Traintest_CR = classification_report(y_test, y_pred)

# overfit
y_pred = logit.predict(X_train)
overfit_CM = confusion_matrix(y_train, y_pred)
overfit_CR = classification_report(y_train, y_pred)

# logit summary
import statsmodels.api as sm
smlogit = sm.Logit(y_train, X_train).fit()
logit_summary = smlogit.summary()


#########################
### validation result ###
#########################
# lower threshold
limiter = lower_limiter
y_prob = list(logit.predict_proba(x_valid)[:,1])
y_pred = []
count =0
for prob in y_prob:
    if prob >= limiter:
        y_pred.append(1)
        count+=1
    else:
        y_pred.append(0)
low_vad_CM = confusion_matrix(y_valid, y_pred)
low_vad_CR = classification_report(y_valid, y_pred)
Dict_low_vad_CR = classification_report(y_valid, y_pred, output_dict=True)

# high threshold
limiter = 0.55 # set to fix value
y_prob = list(logit.predict_proba(x_valid)[:,1])
y_pred = []
count =0
for prob in y_prob:
    if prob >= limiter:
        y_pred.append(1)
        count+=1
    else:
        y_pred.append(0)
high_vad_CM = confusion_matrix(y_valid, y_pred)
high_vad_CR = classification_report(y_valid, y_pred)
Dict_high_vad_CR = classification_report(y_valid, y_pred, output_dict=True)

########################
### excutive summary ###
########################
from sklearn.metrics import recall_score
label_coverage = y_pred.count(1)/len(y_pred)
UCW_coverage = recall_score(y_valid, y_pred)
Excutive_Summary = 'The limiter we adopt is %.2f' % (limiter) + '. ' + 'By covering %.3f labeled as high probability of UCW, we have prepared for %.3f of real UCW' % (label_coverage,UCW_coverage)


########################
### output prediction ##
########################
# return lower_limiter, higher_limiter, Traintest_CM, Traintest_CR, overfit_CM, overfit_CR, 
# low_vad_CM, low_vad_CR, low_vad_CR, low_vad_CR, logit_summary, logit, Excutive_Summary

<ipython-input-31-b5031ab03ce3>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['U_approve2now'] = df['U_approve2now'].fillna(-1)
<ipython-input-31-b5031ab03ce3>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Start_Time'] = pd.to_datetime(df['Start_Time'])


Optimization terminated successfully.
         Current function value: 0.253274
         Iterations 8


### Logistic Regression Summary

In [32]:
print(logit_summary)

                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                56008
Model:                          Logit   Df Residuals:                    55994
Method:                           MLE   Df Model:                           13
Date:                Sun, 13 Jun 2021   Pseudo R-squ.:                 0.05468
Time:                        16:19:13   Log-Likelihood:                -14185.
converged:                       True   LL-Null:                       -15006.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
net_pay                     0.0776      0.003     22.607      0.000       0.071       0.084
prev_CW/SA_rate             2.3231      0.210     11.037      0.000       1.911       2.736


## Classification Report

### Train Test

In [33]:
print(Traintest_CM)
print('\n')
print(Traintest_CR)

[[16780  5358]
 [  848  1018]]


              precision    recall  f1-score   support

           0       0.95      0.76      0.84     22138
           1       0.16      0.55      0.25      1866

    accuracy                           0.74     24004
   macro avg       0.56      0.65      0.55     24004
weighted avg       0.89      0.74      0.80     24004



### Overfit

In [34]:
print(overfit_CM)
print('\n')
print(overfit_CR)

[[39393 12380]
 [ 1850  2385]]


              precision    recall  f1-score   support

           0       0.96      0.76      0.85     51773
           1       0.16      0.56      0.25      4235

    accuracy                           0.75     56008
   macro avg       0.56      0.66      0.55     56008
weighted avg       0.90      0.75      0.80     56008



### Validation
#### <font color = Blue> Lower

In [35]:
print('Applying lower(for Recall) threshhold of {}'.format(lower_limiter))
print('\n')
print(low_vad_CM)
print('\n')
print(low_vad_CR)

Applying lower(for Recall) threshhold of 0.43848450631536495


[[399 513]
 [ 10  78]]


              precision    recall  f1-score   support

           0       0.98      0.44      0.60       912
           1       0.13      0.89      0.23        88

    accuracy                           0.48      1000
   macro avg       0.55      0.66      0.42      1000
weighted avg       0.90      0.48      0.57      1000



#### <font color = Blue> Higher

In [36]:
print('Applying higher(for precision) threshhold of {}'.format(higher_limiter))
print('\n')
print(high_vad_CM)
print('\n')
print(high_vad_CR)

Applying higher(for precision) threshhold of 0.532


[[689 223]
 [ 32  56]]


              precision    recall  f1-score   support

           0       0.96      0.76      0.84       912
           1       0.20      0.64      0.31        88

    accuracy                           0.74      1000
   macro avg       0.58      0.70      0.57      1000
weighted avg       0.89      0.74      0.80      1000



### Excutive Summary

In [37]:
high_label_coverage = (high_vad_CM[0][1]+high_vad_CM[1][1])/1000
high_UCW_coverage = Dict_high_vad_CR['1']['recall']
low_label_coverage = (low_vad_CM[0][1]+high_vad_CM[1][1])/1000
low_UCW_coverage = Dict_low_vad_CR['1']['recall']

print('By setting limiter to %.3f' % (lower_limiter) + '. ' 
      +'We Can prepare for {:.2%} of real UCW, by marking {:.2%} labeled as low probability of UCW。'.format(low_UCW_coverage, low_label_coverage))
print('\n')
print('By setting limiter to %.3f' % (0.55) + '. ' 
      +'We Can prepare for {:.2%} of real UCW, by marking {:.2%} labeled as high probability of UCW。'.format(high_UCW_coverage, high_label_coverage))


By setting limiter to 0.438. We Can prepare for 88.64% of real UCW, by marking 56.90% labeled as low probability of UCW。


By setting limiter to 0.550. We Can prepare for 63.64% of real UCW, by marking 27.90% labeled as high probability of UCW。


# Fit real data in this model

### CM

In [38]:
# set input
real_X = realdata[features_colname]

# concat predicted prob with data
realdata['prob'] = list(logit.predict_proba(real_X)[:,1])

# record when this prediction is ran
from datetime import date
time = str(date.today().year) + '-' + str(date.today().month) + '-' + str(date.today().day)

In [55]:
df = pd.read_sql_query("""

SELECT  sa.id, sa.user_id, sa.shift_id, u."phone", u."name", f.id AS facility_id, sa."withdrawnInfo" -> 'initiator' as withdrawnInfo_value,
sa."status", sa."prevStatus", sa."distance", s."facility_id", "s"."description" AS "shift_description",
"s"."assigned_nurse_id",s."type",
"s"."qualifications" AS "s_qualifications", sa."withdrawnInfo",
"f"."name" AS "facility_name","f"."short_name" AS "f_short_name", f."segmentName", f."areaName", 
timezone('America/Chicago', timezone('UTC', s.start_time)) AS "Start_Time" 
FROM shifts s
INNER JOIN shift_applications sa ON s.id = sa.shift_id
INNER JOIN facilities f ON s.facility_id = f.id
INNER JOIN users u ON sa.user_id = u.id

""", con = connection)

In [56]:
def get_part_of_day(hour):
    return (
        "morning" if 4 < hour <= 12
        else
        "afternoon" if 12 < hour <= 17
        else
        "evening/night" if 18 < hour <= 22
        else
        "overnight"
    )

df['Start_time_of_the_day'] = df.apply(lambda row: get_part_of_day(row['Start_Time'].hour), axis =1)

In [59]:
# read the prediction file
prediction = realdata[['id', 'Start_Time', 'prob']]
validation = prediction.merge(df, on = 'id', how = 'left')

today = date.today()

# convert to datetime for conditonal selection
validation['Start_Time_x'] = pd.to_datetime(validation['Start_Time_x'])

# only select date part of the time
validation['Start_Date'] = validation.apply(lambda row: str(row['Start_Time_x'].date()), axis = 1)

validation['Start_Time'] = validation.apply(lambda row: str(row['Start_Time_x'].time()), axis =1)

# convert to datetime for conditonal selection
validation['Start_Time_x'] = pd.to_datetime(validation['Start_Time_x'])

# only select date part of the time
validation['Start_Time_x'] = validation.apply(lambda row: str(row['Start_Time_x'].date()), axis = 1)

# rename start time
validation = validation.rename(columns={"Start_Time_x": "Start_Time"})

# limit our result to what we want as validation file
validation0 = validation[['id','prob','Start_Time','Start_time_of_the_day','status','type','prevStatus','areaName','segmentName','facility_name','user_id']]

validation0 = validation0.set_index("id")
validation0.columns

validation0.to_csv('pred_{}_Silver_Bullet.csv'.format(time))

In [60]:
# for houston messaging use
validation1 = validation[(validation['areaName'] == 'Houston') & 
           (validation['type'] == 'CNA') & (validation['prob'] >= 0.47)]

# limit our result to what we want as validation file
validation1 = validation1[['id','phone','name','type','Start_Date','Start_Time','facility_name','user_id','prob']]

validation1 = validation1.set_index("id")

validation1.to_excel("list_for_Houston_{}.xlsx".format(time))

In [61]:
# For Houston Plan2 use
pivot_table = validation[(validation['areaName'] == 'Northeast Ohio') & 
           (validation['type'] == 'STNA') & (validation['prob'] >= 0.55)].groupby(["Start_Date",
                                    "Start_time_of_the_day"]).size().reset_index(name='count').set_index("Start_Date")

In [62]:
pivot_table.to_excel("plan2_pred_{}.xlsx".format(time))